## Data Preparation

You should prepare the following things before running this step. Please refer to the `example_data` folder for guidance:

1. **PAR images**
   - the default dimension of model input is [25,128,128,15] where 25 is the number of PAR images, [128,128] is x-y-dimension, 15 is the number of slices

2. **A patient list** that enumerates all your cases.  
   - To understand the expected format, please refer to the file:  
     `example_data/Patient_list/patient_list.xlsx`.

---
## Prediction
The output is a (6,4) matrix, where 6 represents 6 motion parameters (tx, ty, tz, rx, ry, rz), and 4 represents 4 control points (each motion parameter is defined by 5 control points with the first one always 0, here we predict the rest 4).

### Docker environment
1. Please use `docker/docker_tensorflow`, it will build a tensorflow docker


In [1]:
# %%
import os, sys
sys.path.append('/workspace/Documents')

import os
import numpy as np
import nibabel as nb
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input

import HeadCT_MotionCorrection_PARDL.STN.model_STN as model_STN
import HeadCT_MotionCorrection_PARDL.STN.Generator_STN as Generator_STN
import HeadCT_MotionCorrection_PARDL.Data_processing as dp
import HeadCT_MotionCorrection_PARDL.functions_collection as ff
from HeadCT_MotionCorrection_PARDL.Build_lists import Build_list

main_path = '/mnt/camca_NAS/motion_correction/'  # replace with your main path

2025-07-09 17:39:16.724224: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### define trial name and set default parameters

In [2]:
trial_name = 'PAR_model'

CP_num = 5
input_shape = (128,128,15,25)  # 128 for x dim, 128 for y dim, 15 for z dim (15 slices by default), 25 for the number of PAR images

trial_folder = os.path.join(main_path, 'example_data/models', trial_name)
save_folder = os.path.join(main_path, 'example_data/models', trial_name, 'predictions')
ff.make_folder([save_folder])

### set patient list

In [3]:
data_sheet = os.path.join(main_path,'example_data/Patient_list/patient_list.xlsx')
b = Build_list.Build(data_sheet)

_, patient_id_list_test, patient_subid_list_test, random_name_list_test,  start_slice_test, end_slice_test, _, _,  y_motion_param_test, x_par_image_test = b.__build__(batch_list = [0])
  

### create model

In [4]:
# create model architecture:
model_inputs = [Input(input_shape)]
model_outputs=[]
tx, ty, tz, rx, ry, rz = model_STN.get_CNN(nb_filters = [16,32,64,128,256], dimension = 3, CP_num = CP_num)(model_inputs[0])
model_outputs = [tx, ty, tz, rx , ry, rz]

2025-07-09 17:39:21.586599: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-07-09 17:39:21.586796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-07-09 17:39:21.697475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-09 17:39:21.699662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:13:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.49GiB deviceMemoryBandwidth: 1.41TiB/s
2025-07-09 17:39:21.699711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2025-07-09 17:39:21.699804: I tensorflow/stream_executor/platform/defau

### find trained model files

we recommend to predict using multiple models/checkpoints, since we find that averaging/using median across multiple models' results may improve the accuracy.

In [5]:
model_folder = os.path.join(trial_folder, 'models')
model_files = ff.find_all_target_files(['*'],model_folder)

print(len(model_files))

17


### predict

In [8]:
# predict using each model
for j in range(0,len(model_files)):
    f = model_files[j]
    print('loading model from: ', f)
    model= Model(inputs = model_inputs,outputs = model_outputs)
    model.load_weights(f)

    for i in range(0, patient_id_list_test.shape[0]):
        patient_id = patient_id_list_test[i]
        patient_subid = patient_subid_list_test[i]
        random_name = random_name_list_test[i]
        print('processing patient: ', patient_id, patient_subid, random_name)

        save_sub = os.path.join(save_folder, patient_id, patient_subid, random_name,'parameters' , 'slice_' + str(start_slice_test[i]) + '_to_' + str(start_slice_test[i] + 15))
        ff.make_folder([os.path.join(save_folder,patient_id), os.path.join(save_folder,patient_id, patient_subid), os.path.join(save_folder, patient_id, patient_subid, random_name), os.path.dirname(save_sub), save_sub])
        filename = 'model_' + str(j) + '.npy'

        datagen = Generator_STN.DataGenerator(np.asarray([x_par_image_test[i]]),
                                            np.asarray([y_motion_param_test[i]]),
                                            np.asarray([start_slice_test[i]]),
                                            np.asarray([end_slice_test[i]]),
                                            start_slice_sampling = None,
                                            patient_num = 1, 
                                            batch_size =1,
                                            input_dimension = input_shape,
                                            output_vector_dimension = (CP_num - 1,),
                                            shuffle = False,augment = False,)
        
        tx, ty, tz, rx, ry, rz = model.predict_generator(datagen, verbose = 1, steps = 1,)  # unit is "pixel"
        tx = np.reshape(np.asarray(tx),-1) *5
        ty = np.reshape(np.asarray(ty),-1) *5
        tz = np.reshape(np.asarray(tz), -1) *2  # *5 for thin slice, *2 for 2.5mm
        rx = np.reshape(np.asarray(rx),-1)  *5
        ry = np.reshape(np.asarray(ry),-1)  *5
        rz = np.reshape(np.asarray(rz),-1)  *5
        predict = np.reshape(np.concatenate([tx, ty, tz, rx,ry, rz], axis = -1), -1)

        # save
        np.save(os.path.join(save_sub,filename), np.reshape(predict,(6,-1)))



In [10]:
# use all predicted motion parameters to get final prediction via averaging or median
for i in range(0, patient_id_list_test.shape[0]):
    patient_id = patient_id_list_test[i]
    patient_subid = patient_subid_list_test[i]
    random_name = random_name_list_test[i]
    print('processing patient: ', patient_id, patient_subid, random_name)

    start_slice = start_slice_test[i]
    folder = os.path.join(save_folder, patient_id, patient_subid, random_name, 'parameters', 'slice_' + str(start_slice) + '_to_' + str(start_slice + 15))
    files = ff.find_all_target_files(['model_*.npy'], folder)
        
    motion_params = np.zeros((len(files) , 6 , 4))
    for j in range(0, len(files)):
        f = files[j]
        motion_params[j] = np.load(f, allow_pickle=True)

    print('motion parameters shape :', motion_params.shape)

    # average across first axis
    motion_params_avg = np.median(motion_params, axis=0)
    tx = motion_params_avg[0,:]
    ty = motion_params_avg[1,:]
    tz = motion_params_avg[2,:]
    rx = motion_params_avg[3,:]
    ry = motion_params_avg[4,:]
    rz = motion_params_avg[5,:]

    # save averaged motion parameters
    np.save(os.path.join(folder, 'pred_final.npy'), motion_params_avg)

    # # load ground truth
    # gt = np.load(y_motion_param_test[i],allow_pickle = True)
    # gt_tx = gt[0,:][0][1: CP_num]
    # gt_ty = gt[1,:][0][1: CP_num]
    # gt_tz = gt[2,:][0][1: CP_num]  / 2.5
    # gt_rx = np.asarray(gt[3,:][0][1: CP_num])
    # gt_ry= np.asarray(gt[4,:][0][1: CP_num])
    # gt_rz = np.asarray(gt[5,:][0][1: CP_num]) 

    # print('tx: ', tx, ' gt tx:', gt_tx,  ' tx diff: ', np.abs(tx - gt_tx), ' in average: ', np.mean(np.abs(tx - gt_tx)), 'origin max: ',np.max(np.abs(gt_tx)), ' now max: ', np.max(np.abs(tx - gt_tx)))
    # print('ty: ', ty, ' gt ty:', gt_ty,  ' ty diff: ', np.abs(ty - gt_ty), ' in average: ', np.mean(np.abs(ty - gt_ty)), 'origin max: ',np.max(np.abs(gt_ty)), ' now max: ', np.max(np.abs(ty - gt_ty)))
    # print('tz: ', tz, ' gt tz:', gt_tz,  ' tz diff: ', np.abs(tz - gt_tz), ' in average: ', np.mean(np.abs(tz - gt_tz)), 'origin max: ',np.max(np.abs(gt_tz)), ' now max: ', np.max(np.abs(tz - gt_tz)))
    # print('rx: ', rx, ' gt rx:', gt_rx,  ' rx diff: ', np.abs(rx - gt_rx), ' in average: ', np.mean(np.abs(rx - gt_rx)), 'origin max: ',np.max(np.abs(gt_rx)), ' now max: ', np.max(np.abs(rx - gt_rx)))
    # print('ry: ', ry, ' gt ry:', gt_ry,  ' ry diff: ', np.abs(ry - gt_ry), ' in average: ', np.mean(np.abs(ry - gt_ry)), 'origin max: ',np.max(np.abs(gt_ry)), ' now max: ', np.max(np.abs(ry - gt_ry)))
    # print('rz: ', rz, ' gt rz:', gt_rz,  ' rz diff: ', np.abs(rz - gt_rz), ' in average: ', np.mean(np.abs(rz - gt_rz)), 'origin max: ',np.max(np.abs(gt_rz)), ' now max: ', np.max(np.abs(rz - gt_rz)))





processing patient:  MO101701M000006 MO001A000007 random_1
motion parameters shape : (17, 6, 4)
processing patient:  MO101701M000006 MO001A000007 random_1
motion parameters shape : (17, 6, 4)
processing patient:  MO101701M000006 MO001A000007 random_1
motion parameters shape : (17, 6, 4)
